# Rekurentiniai neuroniniai tinklai

Ankstesniame modulyje naudojome turtingas semantines tekstų reprezentacijas ir paprastą linijinį klasifikatorių virš įterpimų. Ši architektūra padeda užfiksuoti agreguotą žodžių prasmę sakinyje, tačiau ji neatsižvelgia į **žodžių tvarką**, nes agregavimo operacija virš įterpimų pašalina šią informaciją iš pradinio teksto. Kadangi šie modeliai negali modeliuoti žodžių tvarkos, jie negali spręsti sudėtingesnių ar dviprasmiškų užduočių, tokių kaip teksto generavimas ar klausimų atsakymas.

Norėdami užfiksuoti teksto sekos prasmę, turime naudoti kitą neuroninių tinklų architektūrą, vadinamą **rekurentiniu neuroniniu tinklu** arba RNN. RNN tinklu mes perduodame savo sakinį per tinklą po vieną simbolį, o tinklas sukuria tam tikrą **būseną**, kurią vėliau perduodame tinklui kartu su kitu simboliu.

Duotai įvesties sekai $X_0,\dots,X_n$, RNN sukuria neuroninių tinklų blokų seką ir treniruoja šią seką nuo pradžios iki pabaigos naudodamas atgalinę sklaidą. Kiekvienas tinklo blokas kaip įvestį gauna porą $(X_i,S_i)$ ir kaip rezultatą sukuria $S_{i+1}$. Galutinė būsena $S_n$ arba išvestis $X_n$ perduodama linijiniam klasifikatoriui, kad būtų gautas rezultatas. Visi tinklo blokai dalijasi tais pačiais svoriais ir yra treniruojami nuo pradžios iki pabaigos per vieną atgalinės sklaidos etapą.

Kadangi būsenos vektoriai $S_0,\dots,S_n$ perduodami per tinklą, jis gali išmokti sekos priklausomybes tarp žodžių. Pavyzdžiui, kai žodis *ne* pasirodo kažkur sekoje, tinklas gali išmokti paneigti tam tikrus elementus būsenos vektoriuje, sukeldamas neigimą.

> Kadangi visų RNN blokų svoriai paveikslėlyje yra bendri, tas pats paveikslėlis gali būti pavaizduotas kaip vienas blokas (dešinėje) su rekursiniu grįžtamojo ryšio ciklu, kuris perduoda tinklo išvesties būseną atgal į įvestį.

Pažiūrėkime, kaip rekurentiniai neuroniniai tinklai gali padėti klasifikuoti mūsų naujienų duomenų rinkinį.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Paprastas RNN klasifikatorius

Naudojant paprastą RNN, kiekvienas pasikartojantis vienetas yra paprastas linijinis tinklas, kuris priima sujungtą įvesties vektorių ir būsenos vektorių, o tada sukuria naują būsenos vektorių. PyTorch šį vienetą atvaizduoja naudodamas `RNNCell` klasę, o tokių ląstelių tinklą - kaip `RNN` sluoksnį.

Norėdami apibrėžti RNN klasifikatorių, pirmiausia pritaikysime įterpimo sluoksnį, kad sumažintume įvesties žodyno dimensiją, o tada virš jo pridėsime RNN sluoksnį:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Pastaba:** Čia naudojame netreniruotą įterpimo sluoksnį dėl paprastumo, tačiau dar geresniems rezultatams galime naudoti iš anksto apmokytą įterpimo sluoksnį su Word2Vec arba GloVe įterpimais, kaip aprašyta ankstesniame skyriuje. Norėdami geriau suprasti, galite pritaikyti šį kodą darbui su iš anksto apmokytais įterpimais.

Mūsų atveju naudosime užpildytą duomenų kaupiklį, todėl kiekvienas paketas turės tam tikrą skaičių užpildytų sekų, kurios bus vienodo ilgio. RNN sluoksnis priims įterpimo tensorių seką ir sugeneruos du išvesties rezultatus:
* $x$ yra RNN ląstelių išvesties seka kiekviename žingsnyje
* $h$ yra galutinė paslėpta būsena paskutiniam sekos elementui

Tada pritaikome pilnai sujungtą linijinį klasifikatorių, kad gautume klasių skaičių.

> **Pastaba:** RNN yra gana sunku treniruoti, nes kai RNN ląstelės yra išskleistos pagal sekos ilgį, sluoksnių, dalyvaujančių atgalinėje propagacijoje, skaičius tampa labai didelis. Todėl reikia pasirinkti mažą mokymosi greitį ir treniruoti tinklą su didesniu duomenų rinkiniu, kad būtų pasiekti geri rezultatai. Tai gali užtrukti gana ilgai, todėl rekomenduojama naudoti GPU.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Ilgalaikė trumpalaikė atmintis (LSTM)

Viena iš pagrindinių klasikinių RNN problemų yra vadinamoji **nykstančių gradientų** problema. Kadangi RNN yra mokomi nuo pradžios iki pabaigos vienu atgalinio sklidimo etapu, jiems sunku perduoti klaidą į pirmuosius tinklo sluoksnius, todėl tinklas negali išmokti ryšių tarp tolimų žodžių. Vienas iš būdų išvengti šios problemos yra įvesti **aiškų būsenos valdymą** naudojant vadinamuosius **vartus**. Yra dvi žinomiausios tokio tipo architektūros: **Ilgalaikė trumpalaikė atmintis** (LSTM) ir **Vartų relės vienetas** (GRU).

![Paveikslėlis, rodantis ilgalaikės trumpalaikės atminties ląstelės pavyzdį](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

LSTM tinklas yra organizuotas panašiai kaip RNN, tačiau yra dvi būsenos, kurios perduodamos iš sluoksnio į sluoksnį: faktinė būsena $c$ ir paslėptas vektorius $h$. Kiekviename vienete paslėptas vektorius $h_i$ yra sujungiamas su įvestimi $x_i$, ir jie kontroliuoja, kas vyksta su būsena $c$ per **vartus**. Kiekvienas vartas yra neuroninis tinklas su sigmoidine aktyvacija (rezultatas intervale $[0,1]$), kurį galima įsivaizduoti kaip bitų kaukę, kai jis dauginamas iš būsenos vektoriaus. Yra šie vartai (iš kairės į dešinę paveikslėlyje aukščiau):
* **užmaršumo vartai** priima paslėptą vektorių ir nustato, kuriuos vektoriaus $c$ komponentus reikia pamiršti, o kuriuos perduoti toliau.
* **įvesties vartai** paima tam tikrą informaciją iš įvesties ir paslėpto vektoriaus bei įterpia ją į būseną.
* **išvesties vartai** transformuoja būseną per tam tikrą linijinį sluoksnį su $\tanh$ aktyvacija, tada pasirenka kai kuriuos jos komponentus naudodami paslėptą vektorių $h_i$, kad sukurtų naują būseną $c_{i+1}$.

Būsenos $c$ komponentus galima įsivaizduoti kaip tam tikrus vėliavėles, kurias galima įjungti ir išjungti. Pavyzdžiui, kai sekoje sutinkame vardą *Alice*, galime manyti, kad jis nurodo moterišką veikėją, ir pakelti vėliavėlę būsenoje, kad sakinyje turime moterišką daiktavardį. Kai toliau sutinkame frazę *ir Tom*, pakelsime vėliavėlę, kad turime daugiskaitinį daiktavardį. Taigi, manipuliuodami būsena, galime, tikėtina, sekti sakinio dalių gramatines savybes.

> **Note**: Puikus šaltinis, padedantis suprasti LSTM vidinę struktūrą, yra šis puikus Christopher Olah straipsnis [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/).

Nors LSTM ląstelės vidinė struktūra gali atrodyti sudėtinga, PyTorch slepia šią įgyvendinimą `LSTMCell` klasėje ir pateikia `LSTM` objektą, skirtą visam LSTM sluoksniui atvaizduoti. Todėl LSTM klasifikatoriaus įgyvendinimas bus gana panašus į paprasto RNN, kurį matėme aukščiau:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Supakuotos sekos

Mūsų pavyzdyje turėjome užpildyti visas mini partijos sekas nulio vektoriais. Nors tai šiek tiek švaisto atmintį, su RNN dar svarbiau, kad papildomos RNN ląstelės yra sukuriamos užpildytoms įvesties reikšmėms, kurios dalyvauja mokyme, tačiau neturi jokios svarbios įvesties informacijos. Būtų daug geriau, jei RNN būtų mokoma tik pagal tikrąją sekos ilgį.

Tam PyTorch įveda specialų užpildytų sekų saugojimo formatą. Tarkime, turime užpildytą įvesties mini partiją, kuri atrodo taip:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Čia 0 reiškia užpildytas reikšmes, o tikrasis įvesties sekų ilgio vektorius yra `[5,3,1]`.

Kad efektyviai mokytume RNN su užpildyta seka, norime pradėti mokymą su pirmąja RNN ląstelių grupe, turinčia didelę mini partiją (`[1,6,9]`), tačiau tada baigti trečios sekos apdorojimą ir tęsti mokymą su mažesnėmis mini partijomis (`[2,7]`, `[3,8]`) ir taip toliau. Taigi, supakuota seka yra pateikiama kaip vienas vektorius – mūsų atveju `[1,6,9,2,7,3,8,4,5]`, ir ilgio vektorius (`[5,3,1]`), iš kurio galime lengvai atkurti pradinę užpildytą mini partiją.

Norėdami sukurti supakuotą seką, galime naudoti funkciją `torch.nn.utils.rnn.pack_padded_sequence`. Visos rekursinės sluoksnių rūšys, įskaitant RNN, LSTM ir GRU, palaiko supakuotas sekas kaip įvestį ir sukuria supakuotą išvestį, kurią galima dekoduoti naudojant `torch.nn.utils.rnn.pad_packed_sequence`.

Kad galėtume sukurti supakuotą seką, turime perduoti ilgio vektorių tinklui, todėl mums reikia kitos funkcijos mini partijoms paruošti:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

Tikrasis tinklas būtų labai panašus į aukščiau pateiktą `LSTMClassifier`, tačiau `forward` perdavimas gaus tiek užpildytą mini partiją, tiek sekų ilgių vektorių. Po įterpimo apskaičiavimo, mes apskaičiuojame supakuotą seką, perduodame ją LSTM sluoksniui ir tada išpakuojame rezultatą atgal.

> **Pastaba**: Iš tikrųjų mes nenaudojame išpakuoto rezultato `x`, nes tolesniuose skaičiavimuose naudojame iš paslėptų sluoksnių gautą išvestį. Todėl šį išpakavimą galima visiškai pašalinti iš šio kodo. Priežastis, kodėl jį čia pateikiame, yra ta, kad jums būtų lengviau modifikuoti šį kodą, jei prireiktų naudoti tinklo išvestį tolimesniuose skaičiavimuose.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **Pastaba:** Galbūt pastebėjote parametrą `use_pack_sequence`, kurį perduodame mokymo funkcijai. Šiuo metu funkcija `pack_padded_sequence` reikalauja, kad ilgio sekos tensorius būtų CPU įrenginyje, todėl mokymo funkcija turi vengti perkelti ilgio sekos duomenis į GPU mokymo metu. Galite peržiūrėti `train_emb` funkcijos įgyvendinimą [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py) faile.


## Dvikryptės ir daugiapakopės RNN

Mūsų pavyzdžiuose visos rekursinės tinklų operacijos vyko viena kryptimi – nuo sekos pradžios iki pabaigos. Tai atrodo natūralu, nes primena būdą, kaip skaitome ar klausomės kalbos. Tačiau daugelyje praktinių atvejų turime atsitiktinę prieigą prie įvesties sekos, todėl gali būti prasminga vykdyti rekursinį skaičiavimą abiem kryptimis. Tokie tinklai vadinami **dvikrypčiais** RNN, ir juos galima sukurti perduodant `bidirectional=True` parametrą RNN/LSTM/GRU konstruktoriui.

Dirbant su dvikrypčiu tinklu, mums reikės dviejų paslėptų būsenų vektorių – po vieną kiekvienai krypčiai. PyTorch koduoja šiuos vektorius kaip vieną dvigubai didesnio dydžio vektorių, kas yra gana patogu, nes paprastai galutinę paslėptą būseną perduodate pilnai sujungtam linijiniam sluoksniui, ir jums tereikia atsižvelgti į šį dydžio padidėjimą kuriant sluoksnį.

Rekursinis tinklas, vienkryptis ar dvikryptis, fiksuoja tam tikrus sekos modelius ir gali juos išsaugoti būsenos vektoriuje arba perduoti į išvestį. Kaip ir konvoliuciniuose tinkluose, galime sukurti kitą rekursinį sluoksnį ant pirmojo, kad fiksuotume aukštesnio lygio modelius, sudarytus iš žemesnio lygio modelių, kuriuos ištraukė pirmasis sluoksnis. Tai veda mus prie **daugiapakopės RNN** sąvokos, kurią sudaro du ar daugiau rekursinių tinklų, kur ankstesnio sluoksnio išvestis perduodama kitam sluoksniui kaip įvestis.

![Vaizdas, rodantis daugiapakopį ilgalaikės-trumpalaikės atminties RNN](../../../../../lessons/5-NLP/16-RNN/images/multi-layer-lstm.jpg)

*Paveikslas iš [šio nuostabaus įrašo](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) autoriaus Fernando López*

PyTorch palengvina tokių tinklų konstravimą, nes jums tereikia perduoti `num_layers` parametrą RNN/LSTM/GRU konstruktoriui, kad automatiškai sukurtumėte kelis rekursijos sluoksnius. Tai taip pat reiškia, kad paslėpto/būsenos vektoriaus dydis proporcingai padidės, ir jums reikės atsižvelgti į tai tvarkant rekursinių sluoksnių išvestį.


## RNN kitiems užduotims

Šiame skyriuje matėme, kad RNN gali būti naudojami sekų klasifikavimui, tačiau iš tiesų jie gali atlikti daug daugiau užduočių, tokių kaip teksto generavimas, mašininis vertimas ir kt. Šias užduotis aptarsime kitame skyriuje.



---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama naudoti profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus interpretavimus, atsiradusius dėl šio vertimo naudojimo.
